# SPOmiAlign Demo: SM (H5AD) to ST (H5AD) Alignment

**任务**：将空间代谢组 (SM) 数据对齐到空间转录组 (ST) 数据，并进行表达量重分配。
**流程**：
1. **Rasterization**: H5AD -> 图像
2. **Alignment**: 图像配准
3. **Visualization**: 叠加检查
4. **Reassignment**: 生成最终 H5AD

In [ ]:
import os
import sys
import cv2
import numpy as np
import matplotlib.pyplot as plt
import scanpy as sc
import seaborn as sns

sys.path.append("../../SPOmiAlign")
from roma import align_and_process_images
from data_preprocessing import rasterize_h5ad_to_image
from reassignment import spomialign_reassignment

%matplotlib inline
plt.rcParams['figure.dpi'] = 100

In [ ]:
# --- 路径配置 ---
DATA_DIR = "../../SPOmiAlign_Repro"
SAVE_DIR = "../../output"
SAVE_PATH = os.path.join(SAVE_DIR, "h5ad_2_h5ad", "sm2st")
os.makedirs(SAVE_PATH, exist_ok=True)

h5ad_img1_path = os.path.join(DATA_DIR, "output_h5ad", "st_withIntensity.h5ad") # Target
h5ad_img2_path = os.path.join(DATA_DIR, "output_h5ad", "sm_withIntensity.h5ad") # Source

# --- 步骤 1: H5AD 转图像 ---
print("🚀 正在栅格化 H5AD...")
Gen_img1_path = os.path.join(SAVE_PATH, "st.png")
Gen_img2_path = os.path.join(SAVE_PATH, "sm.png")

# Target
rasterize_h5ad_to_image(
    input_h5ad=h5ad_img1_path, output_png=Gen_img1_path,
    background="white", point_shape="square", radius=15,
    rotate=0.0, scale=1.0, enhance=False
)
# Source (带预旋转 60度)
rasterize_h5ad_to_image(
    input_h5ad=h5ad_img2_path, output_png=Gen_img2_path,
    background="white", point_shape="square", radius=12,
    rotate=60.0, scale=0.6, enhance=False
)

# 可视化生成的中间图像
fig, ax = plt.subplots(1, 2, figsize=(12, 6))
ax[0].imshow(cv2.imread(Gen_img1_path)[...,::-1]) # BGR->RGB
ax[0].set_title("Target (ST)")
ax[1].imshow(cv2.imread(Gen_img2_path)[...,::-1])
ax[1].set_title("Source (SM) Pre-transformed")
plt.show()

In [ ]:
# --- 步骤 2: 配准 ---
print("🚀 开始配准...")
save_path_alignment = os.path.join(SAVE_PATH, "alignment")
transformed_h5ad_path = os.path.join(save_path_alignment, "transformed.h5ad") 
transformed_h5ad_img_path = os.path.join(save_path_alignment, "transformed_h5ad.png")

# 运行配准核心函数
align_and_process_images(
    img1_path=Gen_img1_path, 
    img2_path=Gen_img2_path, 
    h5ad_path=h5ad_img2_path,  
    method='affine+bspline', 
    output_dir=save_path_alignment,
    rotate=0.0, scale=1.0, # 这里填0因为在栅格化时已经处理了
)

# 将配准后的 H5AD 再次转为图片以供验证
rasterize_h5ad_to_image(
    input_h5ad=transformed_h5ad_path,
    output_png=transformed_h5ad_img_path,
    background="white", point_shape="square", radius=12,
    rotate=0.0, scale=1.0, enhance=False
)
print("✅ 配准完成。")

In [ ]:
# --- 步骤 3: 叠加可视化 ---
print("🎨 生成叠加图...")

target_img = cv2.imread(Gen_img1_path)
# 读取为灰度图作为 Mask
aligned_h5ad_gray = cv2.imread(transformed_h5ad_img_path, cv2.IMREAD_GRAYSCALE)

if target_img is not None and aligned_h5ad_gray is not None:
    # 转回 BGR
    aligned_h5ad_bgr = cv2.cvtColor(aligned_h5ad_gray, cv2.COLOR_GRAY2BGR)
    
    # 创建白色画布
    t_h, t_w = target_img.shape[:2]
    canvas = np.full((t_h, t_w, 3), 255, dtype=np.uint8)
    
    # 放置图像
    h, w = aligned_h5ad_bgr.shape[:2]
    canvas[:min(t_h,h), :min(t_w,w)] = aligned_h5ad_bgr[:min(t_h,h), :min(t_w,w)]

    # 混合叠加
    overlay = cv2.addWeighted(target_img, 0.5, canvas, 0.5, 0)
    
    # 保存并显示
    out_overlay = os.path.join(save_path_alignment, "overlay_check.png")
    cv2.imwrite(out_overlay, overlay)
    
    plt.figure(figsize=(10, 10))
    plt.imshow(cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB))
    plt.title("Overlay Check (Target + Aligned Source)")
    plt.axis('off')
    plt.show()

In [ ]:
# --- 步骤 4: Reassignment ---
print("🧬 执行数据重分配...")

final_h5ad = os.path.join(SAVE_PATH, "sm2st_final_reassigned.h5ad")
map_csv = os.path.join(SAVE_PATH, "sm2st_mapping.csv")

# 请确保你已经有 spomialign_reassignment 函数 (来自 reassignment.py)
spomialign_reassignment(
    s1_h5ad=h5ad_img1_path,         
    s2_h5ad=transformed_h5ad_path,  
    out_h5ad=final_h5ad,
    map_csv=map_csv,
    id_col="id",
    cluster_col="cluster",
    s2_cluster_col=["s2_cluster_col"], 
    scale_by_mapping_factor=True
)
print(f"✨ 最终文件已生成: {final_h5ad}")

In [ ]:
# --- 步骤 5: 最终结果质控 (QC) ---
if os.path.exists(final_h5ad):
    adata = sc.read_h5ad(final_h5ad)
    
    # 1. 距离分布图
    if 'knn_dist' in adata.obs.columns:
        plt.figure(figsize=(8, 5))
        sns.histplot(adata.obs['knn_dist'], bins=50, kde=True)
        plt.title('Distance to Nearest Low-Res Spot')
        plt.xlabel('Distance (pixels)')
        plt.show()
    
    # 2. 空间聚类图 (如果存在)
    if 'cluster' in adata.obs.columns:
        sc.pl.spatial(adata, color='cluster', spot_size=20, title="Reassigned Spatial Clusters")